In [4]:
from difflib import get_close_matches
import csv
from typing import Any, Optional, Set, Dict
from dataclasses import dataclass, field
from pgmpy.readwrite import BIFReader
import numpy as np
from collections import defaultdict, deque
import re
import os

# Parameters
GROUP_ID = 29
ALGORITHM = 'gibbs' # ’ve’ = Variable Elimination, ’gibbs’ = Gibbs Sampling
NETWORK_NAME = './Networks/child.bif'
REPORT = 'Disease' # e.g., Child: ’Disease’
EVIDENCE_LEVEL = 'None' # {None | Little | Moderate} for Child/Insurance
EVIDENCE = 'LowerBodyO2=<5; RUQO2=12+; CO2Report=>=7.5; XrayReport=Asy/Patchy'


@dataclass (frozen=True)
class Node:
    # name is primary key
    name: str
    parents: tuple
    values: tuple
    probability_model: Optional[np.ndarray] = field(default=None, compare=False, repr=False)

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        if isinstance(other, Node):
            return self.name == other.name
        return False

    def __str__(self):
        return f'{self.name}: [{", ".join(self.values)}]'


class Network:
    def __init__(self, nodes: Set[Node] | None = None):
        self.nodes: Set[Node] = nodes or set()
        self.parents: Dict[Node, Set[Node]] = defaultdict(set)   # child -> {parents}
        self.children: Dict[Node, Set[Node]] = defaultdict(set)  # parent -> {children}
        self.by_name: Dict[str, Node] = {}

    def add_node(self, node: Node):
        if node.name not in self.by_name:
            self.by_name[node.name] = node
            self.nodes.add(node)

    def add_edge(self, parent_name: str, child_name: str):
        parent = self.by_name[parent_name]
        child = self.by_name[child_name]
        self.parents[child].add(parent)
        self.children[parent].add(child)

    def markov_blanket(self, node: Node) -> Set[Node]:
        blanket: Set[Node] = self.parents.get(node, set())
        children = self.children.get(node, set())
        blanket.update(children)
        # children's parents excluding self
        for child in children:
            blanket.update(parent for parent in self.parents.get(child, set()) if parent is not node)
        blanket.discard(node)
        return blanket

    def degree(self, node: Node) -> int:
        return len(self.parents[node])+len(self.children[node])

    def topological_order(self):
        degree_in_context = {node:len(self.parents[node]) for node in self.nodes}
        node_queue = deque(sorted(self.nodes, key=lambda node: degree_in_context[node]))

        output = []
        while node_queue:
            current_node = node_queue.pop()
            if degree_in_context[current_node] == 0:
                output.append(current_node)
                for child in self.children[current_node]:
                    degree_in_context[child] -= 1
            else:
                node_queue.appendleft(current_node)

        return output



    def __str__(self):
        return "\n".join(str(n) for n in sorted(self.nodes, key=lambda n: n.name))


class InputReader:
    def __init__(self):
        reader = BIFReader(NETWORK_NAME)
        model = reader.get_model()
        states = reader.get_states()
        net = Network()

        # 1) add nodes with CPDs reshaped to (child, *parents)
        for variable in model.nodes():
            cpd = model.get_cpds(variable)
            parents = tuple(cpd.get_evidence() or tuple())
            child_card = len(states[variable])
            parent_cards = [len(states[p]) for p in parents]

            pm_nd = np.array(cpd.values, dtype=float).reshape(
                (child_card, *parent_cards),
                order="F"  # pgmpy: rightmost parent varies fastest
            )

            net.add_node(Node(
                name=str(variable),
                parents=parents,
                values=tuple(states[variable]),
                probability_model=pm_nd
            ))

        # 2) add edges
        for child in model.nodes():
            cpd = model.get_cpds(child)
            for parent in (cpd.get_evidence() or []):
                net.add_edge(str(parent), str(child))

        self.network = net

        # 3) parse EVIDENCE string
        self.parsed_evidence = {}
        for statement in EVIDENCE.split(";"):
            i = statement.find('=')
            key = statement[:i].strip()
            value = statement[i+1:].strip()
            if key not in self.network.by_name:
                raise Exception("Invalid Evidence " + key)
            self.parsed_evidence[key] = value

class Factor:
    def __init__(self, variables, values):
        """
        variables: list[str] in the order of axes of `values`
        values: np.ndarray with one axis per variable (same order)
        """
        self.variables = list(variables)
        self.values = np.array(values, dtype=float)
        assert self.values.ndim == len(self.variables), \
            "values axes must match number of variables"



    def restrict(self, var, value_index):
        if var not in self.variables:
            return self
        ax = self.variables.index(var)
        # Take the slice and drop that axis
        self.values = np.take(self.values, indices=value_index, axis=ax)
        self.variables.pop(ax)
        return self

    def _align_to(self, all_vars):
        """
        Return a view of self.values whose axes are placed at the positions
        matching all_vars; missing variables are broadcast via singleton axes.
        """
        # Where each of our variables should go in all_vars
        dest_axes = [all_vars.index(v) for v in self.variables]

        # Ensure we have enough axes to move: pad with singleton axes at the end
        arr = self.values
        need = len(all_vars) - arr.ndim
        if need > 0:
            arr = arr.reshape(arr.shape + (1,) * need)

        # Move our current axes (0..k-1) to their destination positions
        arr = np.moveaxis(arr, list(range(len(self.variables))), dest_axes)
        return arr

    def multiply(self, other):
        """
        Pointwise multiply with broadcasting after aligning axes by variable name.
        """
        # Stable union order: keep self order, then add other's new vars
        all_vars = list(dict.fromkeys(self.variables + other.variables))

        A = self._align_to(all_vars)
        B = other._align_to(all_vars)

        # Now A and B have the same ndim and compatible shapes
        prod = A * B
        return Factor(all_vars, prod)

    def sum_out(self, var):
        if var not in self.variables:
            return self
        ax = self.variables.index(var)
        self.values = self.values.sum(axis=ax)
        self.variables.pop(ax)
        return self

    def normalize(self):
        Z = self.values.sum()
        if Z != 0:
            self.values = self.values / Z
        return self


class VESolver:

    @staticmethod
    def _value_index(node, label: str) -> int:
        # exact or case-insensitive
        try:
            return node.values.index(label)
        except ValueError:
            lower_vals = [s.lower() for s in node.values]
            try:
                return lower_vals.index(label.lower())
            except ValueError:
                allowed = ", ".join(node.values)
                raise ValueError(f"Value {label!r} invalid for {node.name}. "
                                 f"Allowed: [{allowed}]")


    # ---------- factor creation ----------
    def _factor_from_node(self, network, node) -> "Factor":
        """
        Reshape node.probability_model (pgmpy CPD values) from
        (child_card, prod(parent_cards)) -> (child_card, *parent_cards)
        following the exact parent order in node.parents.
        """
        child = node.name
        parents = list(node.parents or ())
        variables = [child] + parents
        child_card = len(network.by_name[child].values)
        parent_cards = [len(network.by_name[p].values) for p in parents]

        vals = np.array(node.probability_model, dtype=float).reshape(
            (child_card, *parent_cards),
            order="F"  # pgmpy stores evidence columns in Fortran order
        )
        return Factor(variables, vals)

    def _build_factors(self, network) -> list["Factor"]:
        return [self._factor_from_node(network, n) for n in network.nodes]

    def solve(self, network, query: str, evidence: dict[str, str]):
        # dict of labels: map to indices
        evidence_node_states = {}

        for variable in evidence:
            evidence_node_states[variable] = network.by_name[variable].values.index(evidence[variable])

        # factors
        factors = self._build_factors(network)

        # restrict evidence
        for variable, state in evidence_node_states.items():
            for factor in factors:
                factor.restrict(variable, state)

        elim_order = [node.name for node in network.topological_order() if node.name != query]

        # eliminate
        for eliminating_variable in elim_order:
            bucket = [factor for factor in factors if eliminating_variable in factor.variables]
            if not bucket:
                continue
            new_f = bucket[0]
            for f in bucket[1:]:
                new_f = new_f.multiply(f)
            new_f.sum_out(eliminating_variable)
            # replace bucket with new_f
            factors = [f for f in factors if f not in bucket] + [new_f]

        # multiply remaining & normalize
        result = factors[0]
        for f in factors[1:]:
            result = result.multiply(f)
        result.normalize()

        # result should be a factor over [query] (possibly with size-1 evidence axes)
        # If extra singleton axes remain, drop them until only query remains.
        while result.variables != [query]:
            # remove any size-1 axes by summing (no-op) or squeezing safely
            for variable in list(result.variables):
                if variable != query and result.values.shape[result.variables.index(variable)] == 1:
                    result.sum_out(variable)
                    break
            else:
                # ff we get here, there are still other vars present; multiply must have left something
                break

        print(REPORT+":")
        print(" ".join(network.by_name[REPORT].values))
        print(" ".join([f"{val:.2f}" for val in result.values]))
        return result

from collections import Counter
import numpy as np

from collections import Counter

class GibbsSolver:
    """
    - network.by_name: dict[str, Node]
    - network.parents: dict[Node, set[Node]]
    - network.children: dict[Node, set[Node]]
    - Node: name(str), parents(tuple[str]), values(tuple[str]), probability_model(np.ndarray)
    """

    def __init__(self, iterations=20000, burn_in=5000, thin=1, seed=None, verbose=False):
        assert iterations > 0 and burn_in >= 0 and iterations > burn_in
        assert thin >= 1
        self.iterations = iterations
        self.burn_in = burn_in
        self.thin = thin
        self.rng = np.random.default_rng(seed)
        self.verbose = verbose

    def solve(self, network, report_var: str, evidence: dict[str, str]):
        nodes_by_name = network.by_name                              # str -> Node
        report_node = nodes_by_name[report_var]

        state = {}
        for name, node in nodes_by_name.items():
            if name in evidence:
                state[name] = evidence[name]
            else:
                state[name] = self.rng.choice(node.values)

        non_evidence_nodes = [nodes_by_name[n] for n in nodes_by_name if n not in evidence]

        tally = Counter()
        kept = 0

        for it in range(self.iterations):
            for X_node in non_evidence_nodes:
                X_name = X_node.name
                x_vals = X_node.values
                weights = np.empty(len(x_vals), dtype=float)

                # Score each candidate value (Markov blanket go brrr)
                for i, x in enumerate(x_vals):
                    old_val = state[X_name]
                    state[X_name] = x

                    # p(X=x | Pa(X))
                    w = self._cpd_prob(X_node, state, nodes_by_name)

                    for Y_node in network.children.get(X_node, ()):
                        w *= self._cpd_prob(Y_node, state, nodes_by_name)

                    weights[i] = w if (w > 0 and np.isfinite(w)) else 0.0
                    state[X_name] = old_val 

                s = weights.sum()
                probs = (weights / s) if s > 0 else np.full(len(x_vals), 1.0 / len(x_vals))

                # Samples new value for X
                state[X_name] = self.rng.choice(x_vals, p=probs)

            if it >= self.burn_in and ((it - self.burn_in) % self.thin == 0):
                tally[state[report_var]] += 1
                kept += 1

        if kept == 0:
            tally[state[report_var]] += 1
            kept = 1

        report_values = report_node.values
        counts = np.array([tally[v] for v in report_values], dtype=float)
        probs = counts / counts.sum()

        print(report_var + ":")
        print(" ".join(report_values))
        print(" ".join(f"{p:.2f}" for p in probs))

        return probs.tolist()

    def _cpd_prob(self, node, state, nodes_by_name):
        """
        Return p(node = state[node.name] | parents(node) = state[...])
        """
        pm = node.probability_model
        par_names = node.parents or ()

        # child outcome index
        x_label = state[node.name]
        try:
            x_idx = node.values.index(x_label)
        except ValueError:
            return 0.0

        # parent indices
        par_sizes, par_indices = [], []
        for p in par_names:
            domain = nodes_by_name[p].values
            pv = state[p]
            try:
                idx = domain.index(pv)
            except ValueError:
                return 0.0
            par_sizes.append(len(domain))
            par_indices.append(idx)

        # Multi-dimensional case
        if pm.ndim == 1 + len(par_names):
            try:
                return float(pm[(x_idx, *par_indices)])
            except Exception:
                return 0.0

        # 2-D tabular case
        if pm.ndim == 2:
            if len(par_names) == 0:
                if pm.shape[1] == 1:
                    return float(pm[x_idx, 0])
                if pm.shape[0] == len(node.values):
                    return float(pm[x_idx])
                return 0.0
            try:
                col = 0
                for i, b in zip(reversed(par_indices), reversed(par_sizes)):
                    col = col * b + i
                return float(pm[x_idx, col])
            except Exception:
                return 0.0

        # Unknown layout
        return 0.0



class OutputWriter:
    def __init__(self, output_text: str, network, algorithm: str):
        os.makedirs("outputs", exist_ok=True)
        file_path = f"outputs/{algorithm}_{REPORT}.txt"

        n_vars = len(network.nodes)
        n_edges = sum(len(children) for children in network.children.values())

        with open(file_path, "w", encoding="utf-8") as f:
            f.write(output_text)
            f.write(f"\n{n_vars} variables\n")
            f.write(f"{n_edges} directed edges\n")

class Driver:
    def __init__(self):
        self.reader = InputReader()
        match(ALGORITHM.lower()):
            case "ve":
                self.solver = VESolver()
            case "gibbs":
                self.solver = GibbsSolver(iterations=50000, burn_in=10000, thin=5, seed=GROUP_ID)
            case _:
                raise NotImplementedError

        factor = self.solver.solve(self.reader.network,REPORT,self.reader.parsed_evidence)


        result_text = f"{REPORT}:\n" \
              f"{' '.join(self.reader.network.by_name[REPORT].values)}\n" \
              f"{' '.join(f'{val:.2f}' for val in factor.values if hasattr(factor, 'values')) if hasattr(factor, 'values') else ' '.join(f'{val:.2f}' for val in factor)}\n"

        OutputWriter(result_text, self.reader.network, ALGORITHM)



if __name__ == '__main__':
    Driver()


Disease:
PFC TGA Fallot PAIVS TAPVD Lung
0.03 0.30 0.29 0.27 0.08 0.04
